In [1]:
import  tensorflow as  tf 
import tensorflow.keras   as keras 
from keras.datasets import mnist
import numpy as np 
import  matplotlib.pyplot as plt 
import matplotlib as mpl 

# use Keras to import pre-shuffled MNIST database
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("The MNIST database has a training set of %d examples." % len(X_train))
print("The MNIST database has a test set of %d examples." % len(X_test))


Using TensorFlow backend.


The MNIST database has a training set of 60000 examples.
The MNIST database has a test set of 10000 examples.


In [2]:
display ( X_train.shape ,X_test.shape , X_test.shape , y_test.shape)

(60000, 28, 28)

(10000, 28, 28)

(10000, 28, 28)

(10000,)

In [ ]:
from sklearn.preprocessing import StandardScaler

y_train  = tf.one_hot(y_train  ,depth =10 ) 
y_test = tf.one_hot(y_test  ,depth =10 ) 
x_valid, x_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    X_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

#### 建立模型

In [ ]:
from tensorflow.keras import layers 
from tensorflow.keras import Sequential  
from tensorflow.keras.layers  import Dense ,Flatten ,Dropout  

model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))


In [ ]:
# summarize the model
model.summary()

#### Compile the Model

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

###  Calculate the Classification Accuracy on the Test Set (Before Training)

In [ ]:
# evaluate test accuracy
score = model.evaluate(X_test,y_test ) 
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

In [ ]:
y_test.shape 

#### fit the model 

In [ ]:
import os 
logdir = os.path.join("callbacks" )

if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"mnist_model.h5")


# callbacks 训练过程中
callbacks = [
    keras.callbacks.TensorBoard(logdir),  # 控制台 ，Ui 
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),  #只是保存最好的模型， 默认保存最后的模型
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),  #提前结束
]

epochs = 10 
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)
